# A Bayesian Network to model the relationship between Agriculture and Economy in Ethiopia



## The core part of the implementation is a library termed as pgmpy

In [69]:
import pandas as pd
from pandas_datareader import wb
import pandas_datareader as pdr
import wbdata
from pgmpy.models import BayesianModel
import numpy as np
from pandas import read_csv, DataFrame
from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator
from IPython.core.display import display, HTML
from pgmpy.independencies.Independencies import IndependenceAssertion
from pgmpy.inference import VariableElimination
import time


# The specific values are stored as indicators on the website.

In [42]:
indicators={
    
       "NY.GDP.PCAP.KD.ZG":"gdp",
        'NE.IMP.GNFS.ZS':'imp',
        'NE.EXP.GNFS.ZS':'exp',
    
    'AG.LND.AGRI.ZS':'lnd',
    'AG.LND.AGRI.ZS?':'agr',
    'NV.AGR.TOTL.ZS':'urb',
    'SP.POP.GROW':'pop'
   
    
    }



# Get data from wb

In [85]:
df=wbdata.get_dataframe(indicators,country='ET')

In [86]:
df

,gdp,imp,exp,lnd,agr,urb,pop
date,,,,,,,
2020,3.395270,16.928599,7.089634,NaN,NaN,35.451406,2.541386
2019,5.604370,20.875324,7.939977,NaN,NaN,33.521072,2.579697
2018,4.053940,22.826699,8.372676,33.563258,33.563258,31.111929,2.619969
2017,6.684560,23.474263,7.629369,33.242509,33.242509,33.779281,2.663414
2016,6.509712,27.086972,7.812081,33.057528,33.057528,34.698880,2.708073
...,...,...,...,...,...,...,...
1964,NaN,NaN,NaN,52.979110,52.979110,NaN,2.484148
1963,NaN,NaN,NaN,53.024523,53.024523,NaN,2.454549
1962,NaN,NaN,NaN,52.905540,52.905540,NaN,2.397853


# define nodes and model

In [77]:
nodes = ['gdp','imp','exp','pop','lnd','urb','agr']

In [78]:
model = BayesianModel([('exp', 'gdp'), ('imp','gdp'),('pop','imp'),('urb','lnd'),('lnd','agr'), ('agr','exp')])

In [79]:
years = {"min" : 1960, "max" : 2020} # world bank data is this format

# data pre-process ... 

In [80]:
TIERS_NUM=3 # this is to devide the data into 3 teirs

In [87]:
def boundary_str(start, end, tier):
    return f'{tier}: {start:+0,.2f} to {end:+0,.2f}'

def relabel(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return boundary_str(boundaries[0][0], boundaries[0][1], tier='A')
    elif v >= boundaries[1][0] and v <= boundaries[1][1]:
        return boundary_str(boundaries[1][0], boundaries[1][1], tier='B')
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return boundary_str(boundaries[2][0], boundaries[2][1], tier='C')
    else:
        return np.nan

In [88]:
def get_boundaries(tiers):
    prev_tier = tiers[0]
    boundaries = [(prev_tier[0], prev_tier[prev_tier.shape[0] - 1])]
    for index, tier in enumerate(tiers):
        if index != 0:
            boundaries.append((prev_tier[prev_tier.shape[0] - 1], tier[tier.shape[0] - 1]))
            prev_tier = tier
    return boundaries

In [89]:
new_columns = {}
for i, content in enumerate(df.items()):
    (label, series) = content
    values = np.sort(np.array([x for x in series.tolist() if not np.isnan(x)] , dtype=float))
    if values.shape[0] < TIERS_NUM:
        print(f'Error: there are not enough data for label {label}')
        break
    boundaries = get_boundaries(tiers=np.array_split(values, TIERS_NUM))
    new_columns[label] = [relabel(value, boundaries) for value in series.tolist()]

df = DataFrame(data=new_columns)
df.columns = nodes
df.index = range(years["min"], years["max"] + 1)


# Learning of network parameters

In [92]:
model.cpds = []
model.fit(data=df,
          estimator=BayesianEstimator,
          prior_type="BDeu",
          equivalent_sample_size=10,
          complete_samples_only=False)  

# Key observation in the last cell is the fact that it the line where complete_samples_only=False is where BN is able to learn from incomplete data

In [94]:
print(f'Check model: {model.check_model()}\n')
for cpd in model.get_cpds():
    print(f'CPT of {cpd.variable}:')
    print(cpd, '\n')

Check model: True

CPT of exp:
+--------------------------+------------------------+------------------------+------------------------+
| agr                      | agr(A: +1.23 to +2.62) | agr(B: +2.62 to +2.88) | agr(C: +2.88 to +3.59) |
+--------------------------+------------------------+------------------------+------------------------+
| exp(A: +7.09 to +7.94)   | 0.49122807017543857    | 0.30107526881720437    | 0.3333333333333333     |
+--------------------------+------------------------+------------------------+------------------------+
| exp(B: +7.94 to +11.64)  | 0.3333333333333333     | 0.30107526881720437    | 0.3333333333333333     |
+--------------------------+------------------------+------------------------+------------------------+
| exp(C: +11.64 to +16.69) | 0.17543859649122806    | 0.3978494623655914     | 0.3333333333333333     |
+--------------------------+------------------------+------------------------+------------------------+ 

CPT of gdp:
+------------------

## At this point the model is ready to go. That means factorizations, conditional dependencies and the priors, have already been learned by the algorithm. Then from here, we can make all types of queries that we think are of economic value. 

## For example to see what factor is dependant on what, we can query to see all the independencies as ... 

In [95]:
model.get_independencies()

(agr ⟂ imp, pop)
(agr ⟂ imp | pop)
(agr ⟂ imp, pop, urb | lnd)
(agr ⟂ imp, pop | urb)
(agr ⟂ pop | imp)
(agr ⟂ imp, pop, gdp | exp)
(agr ⟂ imp, urb | pop, lnd)
(agr ⟂ imp | pop, urb)
(agr ⟂ imp, gdp | pop, exp)
(agr ⟂ imp, pop | urb, lnd)
(agr ⟂ urb | lnd, gdp)
(agr ⟂ pop, urb | imp, lnd)
(agr ⟂ imp, pop, urb, gdp | lnd, exp)
(agr ⟂ pop | imp, urb)
(agr ⟂ imp, pop, gdp | urb, exp)
(agr ⟂ pop | imp, gdp)
(agr ⟂ imp, pop | gdp, exp)
(agr ⟂ pop, gdp | imp, exp)
(agr ⟂ imp | urb, pop, lnd)
(agr ⟂ urb | pop, lnd, gdp)
(agr ⟂ urb | imp, pop, lnd)
(agr ⟂ imp, urb, gdp | pop, lnd, exp)
(agr ⟂ imp, gdp | pop, urb, exp)
(agr ⟂ imp | pop, gdp, exp)
(agr ⟂ gdp | imp, pop, exp)
(agr ⟂ pop | urb, imp, lnd)
(agr ⟂ imp, pop, gdp | urb, lnd, exp)
(agr ⟂ pop, urb | imp, lnd, gdp)
(agr ⟂ imp, pop, urb | lnd, exp, gdp)
(agr ⟂ pop, urb, gdp | imp, lnd, exp)
(agr ⟂ pop | imp, urb, gdp)
(agr ⟂ imp, pop | urb, exp, gdp)
(agr ⟂ pop, gdp | imp, urb, exp)
(agr ⟂ pop | imp, gdp, exp)
(agr ⟂ imp, gdp | urb, pop, l

## The last cell alone shows a lot of valable information for an economist or a policy maker

In [98]:
model.local_independencies('gdp')

(gdp ⟂ agr, lnd, pop, urb | imp, exp)

# print markov blankets

In [103]:
model.get_markov_blanket('pop')

['imp']

# read indepndeces 

In [74]:
model.local_independencies('imp')

(imp ⟂ agr, lnd, urb, exp | pop)

In [67]:
model.get_markov_blanket('lnd')

['agr', 'urb']

# Inference 

In [105]:
infer = VariableElimination(model)

In [113]:
print(infer.query(variables=['gdp'], evidence={'exp':1}))

Eliminating: pop: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 271.84it/s]

+-------------------------+------------+
| gdp                     |   phi(gdp) |
+=========================+============+
| gdp(A: -13.94 to +0.05) |     0.1754 |
+-------------------------+------------+
| gdp(B: +0.05 to +6.68)  |     0.3130 |
+-------------------------+------------+
| gdp(C: +6.68 to +10.41) |     0.5115 |
+-------------------------+------------+


In [115]:
print(infer.query(variables=['gdp']))

Eliminating: exp: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 557.65it/s]

+-------------------------+------------+
| gdp                     |   phi(gdp) |
+=========================+============+
| gdp(A: -13.94 to +0.05) |     0.1960 |
+-------------------------+------------+
| gdp(B: +0.05 to +6.68)  |     0.4088 |
+-------------------------+------------+
| gdp(C: +6.68 to +10.41) |     0.3952 |
+-------------------------+------------+


# assetinos can also be made

In [117]:
IndependenceAssertion('pop', 'lnd', 'gdp')

(pop ⟂ lnd | gdp)